Install dependencies

In [ ]:
!python -m pip install torch torchvision torchaudio
!python -m pip install ultralytics roboflow matplotlib

Start the training

In [1]:
# 0. Preliminary Setup
!pip install ultralytics roboflow

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# 1. GPU Check
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"Current Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

# 2. Roboflow Dataset Download
rf = Roboflow(api_key="YOUR_ROBOFLOW_API_KEY")
project = rf.workspace("your-workspace").project("your-project")
version = project.version(1)  # Choose your dataset version
dataset = version.download("yolov8")

# 3. Configuration
CONFIG = {
    'model': 'yolov8m.pt',  # Choose model size: n, s, m, l, x
    'data': f"{dataset}/data.yaml",
    'epochs': 100,
    'batch': 16,  # Adjust based on GPU memory
    'imgsz': 640,
    'patience': 20,
    'device': 0  # GPU index
}

# 4. Training
model = YOLO(CONFIG['model'])
results = model.train(
    data=CONFIG['data'],
    epochs=CONFIG['epochs'],
    batch=CONFIG['batch'],
    imgsz=CONFIG['imgsz'],
    patience=CONFIG['patience'],
    device=CONFIG['device'],
    
    # Optional advanced configurations
    optimizer='Adam',
    lr0=0.001,
    lrf=0.1,
    momentum=0.9,
    weight_decay=0.0005,
    warmup_epochs=3,
    warmup_momentum=0.8,
    
    # Augmentations
    augment=True,
    hsv_h=0.015,  # Image HSV-Hue augmentation
    hsv_s=0.7,   # Image HSV-Saturation augmentation
    hsv_v=0.4,   # Image HSV-Value augmentation
    degrees=45,  # Image rotation (+/- deg)
    translate=0.1,  # Image translation (+/- fraction)
    scale=0.5,   # Image scale (+/- gain)
    shear=0.0,   # Image shear (+/- deg)
    perspective=0.0,  # Image perspective
    flipud=0.5,  # Image flip up-down
    fliplr=0.5,  # Image flip left-right
    mosaic=1.0,  # Image mosaic
    mixup=0.0,   # Image mixup
)

# 5. Validation
val_results = model.val(data=CONFIG['data'])

# 6. Export (Optional)
model.export(format='onnx')  # Export to ONNX
model.export(format='torchscript')  # Export to TorchScript

# 7. Plotting Results
import matplotlib.pyplot as plt

# Plot training results
plt.figure(figsize=(15,5))
plt.subplot(131)
plt.plot(results.results_dict['train/box_loss'], label='Box Loss')
plt.title('Training Box Loss')
plt.subplot(132)
plt.plot(results.results_dict['train/cls_loss'], label='Class Loss')
plt.title('Training Class Loss')
plt.subplot(133)
plt.plot(results.results_dict['train/dfl_loss'], label='DFL Loss')
plt.title('Training DFL Loss')
plt.tight_layout()
plt.show()

# 8. Performance Metrics
print("\nValidation Metrics:")
for metric, value in val_results.results_dict.items():
    print(f"{metric}: {value}")

/bin/bash: line 1: /home/larizmen/.local/bin/pip: cannot execute: required file not found


ModuleNotFoundError: No module named 'torch'